# Genetic Algorithms vs. Sumulated Annealing 
### Regression Approximation

By Max Wiesner

In [ ]:
%matplotlib inline
import numpy as np
from tqdm import tqdm
from curveFitting import one_dimensional_curve_fitting_test
from IPython.display import display, HTML
import pprint
import pandas as pd
import math

tests = [
    [lambda x: 0.2*math.exp(x/4.0) -  math.sin(2*x), (-10.0, 10.0), 25],
    [lambda x: math.sin(2*x) - math.cos(3*x-1) + 2*math.sin(x/2+1.0), (-15.0, 15.0), 60],
    [lambda x: math.exp(math.sin(2*x)) - math.exp(math.cos(3*x)), (-15.0, 15.0), 60],
    [lambda x: 0.05*x**2 - 0.5*x + 5.0 * math.sin(3*x) , (-15.0, 15.0), 60],
    [lambda x: 0.05*x**2 - 0.5*math.sqrt(x) + 5.0 * math.sin(3*x) , (0.0, 15.0), 30]
]

each_test = 3
i_std = 0.5
p_std = 0.5
GA_criteria = np.array([[i_std*1000, p_std*40], [i_std*500, p_std*80], [i_std*250, p_std*160]]).astype(int)
SA_criteria = np.array([[i_std*10000, p_std*40], [i_std*5000, p_std*80], [i_std*2500, p_std*160]]).astype(int)

results = []
test_instances = [{'results': [[-float('inf')]*5 for i in range(3)],
                   'all': [[0]*5 for i in range(3)],
                   'conditions': criteria} for criteria in [GA_criteria, SA_criteria]]

with tqdm(total = each_test*len(GA_criteria)*len(test_instances)*len(tests)) as pbar:
    for iteration in range(each_test):
        for curr_algorithm in test_instances:
            for curr_condition_index in range(len(curr_algorithm['conditions'])):
                for curr_test_index in range(len(tests)):
                    all_data_results = one_dimensional_curve_fitting_test(tests[curr_test_index][0], tests[curr_test_index][1], 
                        tests[curr_test_index][2], curr_algorithm['conditions'][curr_condition_index][0], curr_algorithm['conditions'][curr_condition_index][1])
                    
                    if all_data_results[1] > curr_algorithm['results'][curr_condition_index][curr_test_index]:
                        curr_algorithm['results'][curr_condition_index][curr_test_index] = all_data_results[1]
                        curr_algorithm['all'][curr_condition_index][curr_test_index] = all_data_results
                    pbar.update(1)

 18%|█▊        | 16/90 [05:17<1:06:24, 53.85s/it]

In [ ]:
css = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(css))

title_specs = {
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '18px')
    ]
}

def highlight_last_max(data, colormax='antiquewhite', colormaxlast='lightgreen'):

    colormax_attr = f'background-color: {colormax}'
    colormaxlast_attr = f'background-color: {colormaxlast}'
    max_value = data.max()
    is_max = [colormax_attr if v == max_value else '' for v in data]
    is_max[len(data) - list(reversed(data)).index(max_value) -  1] = colormaxlast_attr
    return is_max

    
def create_data_frames():
    dfs = [None]*2
    names = ['Genetic Algorithm', 'Simulated Annealing Algorithm']
     
    for ind in range(len(names)):
        df = pd.DataFrame(np.array( [[test_instances[ind]['results'][i][j] for j in range(5)] for i in range(3)]), index = [f'{crit[0]}, {crit[1]}' for crit in GA_criteria], 
                                   columns = [f'Function {i}' for i in range(1, 6)])
        
        df.style.set_table_attributes("style='display:inline'").set_caption(f'{names[ind]} - Regression Estimation')
        df.loc['Fitness Avg by Function']= df.mean(numeric_only=True, axis=0)
        df.loc[:,'Fitness Avg by Criteria'] = df.mean(numeric_only=True, axis=1)
        pd.options.display.float_format = '{:,.3f}'.format        
        df = df.rename_axis('Criteria: Num of Iterations, Population Size')
        
        df['Run Time Avg by Criteria'] = [np.mean([test_instances[0]['all'][i][j][3] for j in range(5)]) for i in range(3)] + [None]
        df.loc['Run Time Avg by Function'] = [np.mean([test_instances[0]['all'][i][j][3] for i in range(3)]) for j in range(5)] + \
            [None, np.mean([np.mean([test_instances[0]['all'][i][j][3] for i in range(3)]) for j in range(5)])]
        
        df = df.style.apply(highlight_last_max, subset = pd.IndexSlice[[f'{crit[0]}, {crit[1]}' for crit in GA_criteria], [f'Function {i}' for i in range(1, 6)]], axis = 0) \
            .set_table_attributes("style='display:inline'") \
            .set_caption(f'{names[ind]} - Regression Estimation') \
            .set_table_styles([title_specs]) \
            .apply(highlight_last_max, subset = pd.IndexSlice["Fitness Avg by Function":"Run Time Avg by Function", \
                                            [f'Function {i}' for i in range(1, 6)]], axis = 1, colormaxlast = 'lightblue') \
            .apply(highlight_last_max, subset = pd.IndexSlice[ [f'{crit[0]}, {crit[1]}' for crit in GA_criteria], \
                                            ['Fitness Avg by Criteria', 'Run Time Avg by Criteria']], axis = 0, colormaxlast = 'lightblue') \
            .apply(highlight_last_max, subset = pd.IndexSlice["Fitness Avg by Function", ['Fitness Avg by Criteria']], axis = 0, colormaxlast = '#ffcccb')  \
            .apply(highlight_last_max, subset = pd.IndexSlice["Run Time Avg by Function", ['Run Time Avg by Criteria']], axis = 0, colormaxlast = '#ffcccb')  
        dfs[ind] = df
        
    return dfs
    
dfs = create_data_frames()
print('\n')
display(dfs[0])
print('\n\n')
display(dfs[1])
print('\n')

In [ ]:
def generate_and_plot(data):
    plt.figure(1)
    x_values = [x_value for ([x_value], _) in data]
    plt.plot(x_values, [y for (_,y) in data],'x')
    test_xvalues = sorted([x for [x] in test_points])
    result = [best_expr.eval({'x':x_value}) for x_value in test_xvalues ]
    gTruth = [lambda_fun(x_value) for x_value in test_xvalues ]
    plt.plot(test_xvalues, result, 'r-',label='ga_fit')
    plt.plot(test_xvalues, gTruth, 'g-', label='ground-truth')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.figure(2)
    plt.plot(range(len(stats)), [st for st in stats], 'b-')
    plt.xlabel('Iters')
    plt.ylabel('Max Fitness')
    plt.plot(range(len(stats)), [(st[1] if st[1] > -100 else -100) for st in stats], 'r--')
    plt.show()

Write a brief analysis of your results. Some questions to answer:
- Which problem ran the fastest? Why do you think this is the case?
- Which problem ended with the highest fitness? Why do you think this is the case?
- How similar were the running times across the 5 repetitions?
- How similar were the final fitness scores across the 5 repetitions?
- Was there anything else interesting in your results?

`Your answer here.`

## Describe Your Implementation and Reflect

Some questions to answer in your write-up:
- What was your strategy to solve this problem?
- How did you structure your `GASolver` class?
- What other files or methods did you change?
- What was the most difficult part of the project?
- What was something surprising you found?

Please provide some elaboration with your answers. When describing your implementation, remember that we have not watched you solve this problem so you will need to give more details than you think is necessary.

`Your answer here.`

## Going Above and Beyond

Describe the extra changes you implemented. Give some details, perhaps a short paragraph for each major change.

`Your answer here.`

Describe the analysis you did and provide your results in the form of a figure or a table.

`Your answer here.`

Reflect on your work for this section. What was interesting, or what did you learn? A short paragraph here is fine.

`Your answer here.`